In [10]:
import sys
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install bs4

Processing c:\users\datka\appdata\local\pip\cache\wheels\a0\b0\b2\4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472\bs4-0.0.1-cp37-none-any.whl


In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [183]:
class indeed_bot():
    
    # Default bot parameters
    def __init__(self,driver = None, job_title = 'web developer', location = 'texas', silent = False, timeout = 5):
        self.driver = driver
        self.job_title = job_title
        self.location = location
        self.silent = silent
        self.timeout= timeout
    
    # Initialize Selenium webdriver
    def start(self):
        from selenium import webdriver
        from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
        from selenium.webdriver.firefox.options import Options
        if not self.silent:
            print('Starting Selenium Firefox Webdriver...')
        caps = DesiredCapabilities().FIREFOX
        caps['pageLoadStrategy'] = 'normal'
        options = Options()
        options.headless = False
        self.driver = webdriver.Firefox(executable_path = 'c:/geckodriver/geckodriver.exe', options=options, capabilities=caps)
        self.driver.set_page_load_timeout(self.timeout)
        if not self.silent:
            print('Done.')
            
    # Search for X job in Y location
    def search(self, job_title, location):
        # construct the url
        url = 'https://www.indeed.com/jobs?q={}&l={}'.format(job_title.replace(' ','+'), location.capitalize())
        if not self.silent:
            print('Opening:', url)
        self.job_title = job_title
        self.location = location
        self.driver.get(url)
        
    # Close the pop-up that does not let us interact with the page
    def close_popup(self):
        try:
            # Find the "no, thanks" link, click on it.
            no_thanks = self.driver.find_element_by_link_text('No, thanks')
            no_thanks.click()
        except:
            pass
    
    # Get n jobs from current page
    def fetch_from_page(self, n = 0):
        from bs4 import BeautifulSoup
        import pandas as pd
        import time
        xp = "//div[contains(@class, 'jobsearch-SerpJobCard unifiedRow row result clickcard')]"
        # find job containers on current page
        job_elements = self.driver.find_elements_by_xpath(xp)
        if not self.silent:
            print('Jobs on current page:', len(job_elements))
        
        if n!=0:
            job_elements = job_elements[0:min(len(job_elements)-1, n)]
        
        j = []
        # click on every job posting on current page
        for element in job_elements:
            time.sleep(1)
            self.close_popup()

            element.click()

            # job description
            src = self.driver.page_source
            soup = BeautifulSoup(src)
            description = soup.find('div', {"id": "vjs-desc"}) 


            # company and locations
            cl = element.find_elements_by_class_name('sjcl')

            if len(cl)>0:
                company_location = cl[0].text
            else:
                company_location = None    


            # title
            t = element.find_elements_by_class_name('title')
            if len(cl)>0:
                title = t[0].text
            else:
                title = None

            # salary
            s = element.find_elements_by_class_name('salarySnippet.holisticSalary')
            if len(s)>0:
                salary = s[0].text
            else:
                salary = None
            if not self.silent:
                print(len(j),title)
            try:    
                #in case desciprtion did not load
                j.append((title, company_location, salary, description.get_text()))
            except:
                pass
            
            JOBS = pd.DataFrame(j, columns = ['title', 'company_location','salary','description'])
            JOBS_CL = JOBS['company_location'].str.split('\n', expand = True)
            JOBS_CL.columns = ['company/rating', 'location']
            JOBS = pd.concat([JOBS.drop(columns = 'company_location'), JOBS_CL], axis = 1)
        return JOBS
    
    # Navigate to next page and wait for it to load.
    def next_page(self):
        import time
        if not self.silent:
            print('Navigating to next page')
        next_link = self.driver.find_elements_by_partial_link_text('Next')
        if len(next_link)>0:
            next_link[-1].click()
        time.sleep(self.timeout)
        
    # Get a total of N jobs    
    def fetch(self, n = 100):
        import pandas as pd
        n_current = 0
        j_list = []
        while n_current < n:
            j = self.fetch_from_page()
            j_list.append(j)
            n_current += len(j)
            self.next_page()
        return pd.concat(j_list).reset_index(drop = True).head(n)
    
    # Shut down the driver
    def quit(self):
        try:
            print('Shutting down.')
            self.driver.quit()
        except:
            pass

In [184]:
bot = indeed_bot()

In [185]:
bot.start()

Starting Selenium Firefox Webdriver...
Done.


In [186]:
bot.search('data scientist', 'texas')

Opening: https://www.indeed.com/jobs?q=data+scientist&l=Texas


In [187]:
X = bot.fetch(25)

Jobs on current page: 14
0 Data Scientist - Immediate need
1 Data Integration and Analytics Student Intern
2 Data Scientist
3 Data Scientist
4 Senior Data & Applied Scientist
5 Data Scientist (Graph DB, Python Developer)
6 Data Scientist
7 Remote Sensing Geospatial Data Scientist
8 Data Scientist, Product Analytics
9 Senior Data Scientist
10 Data Scientist
11 Data Scientist
12 AppleCare Data Scientist
13 Artificial Intelligence, Consultant - Applied Artificial Int...
Navigating to next page
Jobs on current page: 12
0 Artificial Intelligence, Senior Consultant - Applied Artific...
1 Data Scientist
2 Artificial Intelligence Researcher
3 Data Scientist
4 Data Scientist III
5 App Store & Arcade Data Scientist, Apple Media Products
6 Data Scientist
7 Data Scientist
8 Business Intelligence Consultant/Data Scientist, Junior
9 Remote Sensing Data Scientist
10 Data Scientist
11 Data Scientist
Navigating to next page


In [188]:
X

,title,salary,description,company/rating,location
0,Data Scientist - Immediate need,$45 - $48 an hour,"Our client, which is a large communications co...",MNR IT Solutions LLC,"Plano, TX 75075"
1,Data Integration and Analytics Student Intern,$13.50 an hour,WHO WE ARE:\n\n\nThe Teacher Retirement System...,Teacher Retirement System of Texas 2.8,"Austin, TX 78701 (Downtown area)"
2,Data Scientist,"$80,000 - $120,000 a year",We are looking for data scientist that prefer ...,Robert Half 3.9,"Houston, TX 77072 (Sugarland area)"
3,Data Scientist,None,"The Zebra, named a Best Place to Work in Austi...",The Zebra 4.0,"Austin, TX 78701 (Downtown area)"
4,Senior Data & Applied Scientist,None,Senior Data & Applied Scientist\nDo you have a...,Microsoft 4.2,"Austin, TX 78758 (North Austin area) • Remote ..."
5,"Data Scientist (Graph DB, Python Developer)",None,"Company Culture/Introduction: BlockTrace, is a...","BlockTrace, LLC","Leander, TX 78641"
6,Data Scientist,"$6,000 a month",Energy Cognito is a specialized consulting/sol...,Energy Cognito,"Austin, TX"
7,Remote Sensing Geospatial Data Scientist,None,Job Posting Title:\nRemote Sensing Geospatial ...,University of Texas at Austin 4.3,"Austin, TX 78712 (University of Texas area) • ..."
8,"Data Scientist, Product Analytics",None,We’re looking for Data Scientists to work on o...,Facebook 4.2,"Austin, TX"
9,Senior Data Scientist,"$100,000 - $150,000 a year","Houston TXRevolutionizing BI analytics, Zdaly ...",Zdaly,"Houston, TX 77042 (Westchase area) • Remote wo..."


In [189]:
bot.quit()

Shutting down.
